In [24]:
pip install requests dash dash-core-components dash-html-components

In [33]:
import requests
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Function to fetch weather data from OpenWeatherMap API
def get_weather_data(city):
    api_key = '6bdd8fb1ac7e068de8fd9bd8a437a47b'  # Replace with your actual API key
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'  # units=metric for Celsius
    response = requests.get(url)
    data = response.json()
    return data

# Create Dash application
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    html.H1("Weather Dashboard", style={'text-align': 'center'}),
    dcc.Input(id='city-input', type='text', placeholder='Enter city name', debounce=True),
    html.Div(id='weather-info', style={'margin-top': '20px'})
])

# Define callback function to update weather information based on user input
@app.callback(
    Output('weather-info', 'children'),
    [Input('city-input', 'value')]
)
def update_weather_info(city):
    if city:
        data = get_weather_data(city)
        if 'main' in data:
            temperature = data['main']['temp']
            weather_description = data['weather'][0]['description'].capitalize()
            humidity = data['main']['humidity']
            wind_speed = data['wind']['speed']
            return html.Div([
                html.H3(f"Weather in {city}:"),
                html.P(f"Temperature: {temperature}°C"),
                html.P(f"Weather Description: {weather_description}"),
                html.P(f"Humidity: {humidity}%"),
                html.P(f"Wind Speed: {wind_speed} m/s")
            ])
        else:
            return html.Div("City not found")

# Run the Dash application
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>